In [1]:
import soynlp
from soynlp.hangle import compose, decompose

class Lemmatizer:
    def __init__(self, stems, predefined=None):
        self._stems = stems
        self._predefined = {}
        if predefined:
            self._predefined.update(predefined)

    def is_stem(self, w): return w in self._stems

    def lemmatize(self, word):
        raise NotImplemented

    def candidates(self, word):
        candidates = set()
        for i in range(1, len(word) + 1):
            l = word[:i]
            r = word[i:]
            candidates.update(self._candidates(l, r))
        return candidates

    def _candidates(self, l, r):
        candidates = set()
        if self.is_stem(l):
            candidates.add((l, r))

        l_last = decompose(l[-1])
        l_last_ = compose(l_last[0], l_last[1], ' ')
        r_first = decompose(r[0]) if r else ('', '', '')
        r_first_ = compose(r_first[0], r_first[1], ' ') if r else ' '
        
        # ㄷ 불규칙 활용: 깨닫 + 아 -> 깨달아
        if l_last[2] == 'ㄹ' and r_first[0] == 'ㅇ':
            l_stem = l[:-1] + compose(l_last[0], l_last[1], 'ㄷ')
            if self.is_stem(l_stem):
                candidates.add((l_stem, r))

        # 르 불규칙 활용: 굴 + 러 -> 구르다
        if (l_last[2] == 'ㄹ') and (r_first_ == '러' or (r_first_ == '라')):
            l_stem = l[:-1] + compose(l_last[0], l_last[1], ' ') + '르'
            r_canon = compose('ㅇ', r_first[1], r_first[2]) + r[1:]
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

        # ㅂ 불규칙 활용: 더러 + 워서 -> 더럽다
        if (l_last[2] == ' ') and (r_first_ == '워' or r_first_ == '와'):
            l_stem = l[:-1] + compose(l_last[0], l_last[1], 'ㅂ')
            r_canon = compose('ㅇ', 'ㅏ' if r_first_ == '와' else 'ㅓ', r_first[2]) + r[1:]
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

#         # 어미의 첫글자가 종성일 경우 (-ㄴ, -ㄹ, -ㅂ, -ㅅ)
#         # 입 + 니다 -> 입니다
        if l_last[2] == 'ㄴ' or l_last[2] == 'ㄹ' or l_last[2] == 'ㅂ' or l_last[2] == 'ㅆ':
            l_stem = l[:-1] + compose(l_last[0], l_last[1], ' ')
            r_canon = l_last[2] + r
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

#         # ㅅ 불규칙 활용: 부 + 었다 -> 붓다
#         # exception : 벗 + 어 -> 벗어
        if (l_last[2] == ' ' and l[-1] != '벗') and (r_first[0] == 'ㅇ'):
            l_stem = l[:-1] + compose(l_last[0], l_last[1], 'ㅅ')
            if self.is_stem(l_stem):
                candidates.add((l_stem, r))

        # 우 불규칙 활용: 똥퍼 + '' -> 똥푸다
        if l_last_ == '퍼':
            l_stem = l[:-1] + '푸'
            r_canon = compose('ㅇ', l_last[1], l_last[2]) + r
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

        # 우 불규칙 활용: 줬 + 어 -> 주다
        if l_last[1] == 'ㅝ':
            l_stem = l[:-1] + compose(l_last[0], 'ㅜ', ' ')
            r_canon = compose('ㅇ', 'ㅓ', l_last[2]) + r
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

        # 오 불규칙 활용: 왔 + 어 -> 오다
        if l_last[1] == 'ㅘ':
            l_stem = l[:-1] + compose(l_last[0], 'ㅗ', ' ')
            r_canon = compose('ㅇ', 'ㅏ', l_last[2]) + r
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

        # ㅡ 탈락 불규칙 활용: 꺼 + '' -> 끄다 / 텄 + 어 -> 트다
        if (l_last[1] == 'ㅓ' or l_last[1] == 'ㅏ'):
            l_stem = l[:-1] + compose(l_last[0], 'ㅡ', ' ')
            r_canon = compose('ㅇ', l_last[1], l_last[2]) + r
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

        # 거라, 너라 불규칙 활용
        # '-거라/-너라'를 어미로 취급하면 규칙 활용
        # if (l[-1] == '가') and (r and (r[0] == '라' or r[:2] == '거라')):
        #    # TODO

        # 러 불규칙 활용: 이르 + 러 -> 이르다
        # if (r_first[0] == 'ㄹ' and r_first[1] == 'ㅓ'):
        #     if self.is_stem(l):
        #         # TODO

        # 여 불규칙 활용
        # 하 + 였다 -> 하 + 았다 -> 하다: '였다'를 어미로 취급하면 규칙 활용

        # 여 불규칙 활용 (2)
        # 했 + 다 -> 하 + 았다 / 해 + 라니깐 -> 하 + 아라니깐 / 했 + 었다 -> 하 + 았었다
        if l_last[0] == 'ㅎ' and l_last[1] == 'ㅐ':
            l_stem = l[:-1] + '하'
            r_canon = compose('ㅇ', 'ㅏ', l_last[2]) + r
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

        # ㅎ (탈락) 불규칙 활용
        # 파라 + 면 -> 파랗다
        if (l_last[2] == ' ' or l_last[2] == 'ㄴ' or l_last[2] == 'ㄹ' or l_last[2] == 'ㅂ' or l_last[2] == 'ㅆ'):
            l_stem = l[:-1] + compose(l_last[0], l_last[1], 'ㅎ')
            r_canon = r if l_last[2] == ' ' else l_last[2] + r
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

        # ㅎ (축약) 불규칙 할용
        # 시퍼렜 + 다 -> 시퍼렇다, 파랬 + 다 -> 파랗다, 파래 + '' -> 파랗다
        if (l_last[1] == 'ㅐ') or (l_last[1] == 'ㅔ'):
            # exception : 그렇 + 아 -> 그래
            if len(l) >= 2 and l[-2] == '그' and l_last[0] == 'ㄹ':
                l_stem = l[:-1] + '렇'
            else:
                l_stem = l[:-1] + compose(l_last[0], 'ㅓ' if l_last[1] == 'ㅔ' else 'ㅏ', 'ㅎ')
            r_canon = compose('ㅇ', 'ㅓ' if l_last[1] == 'ㅔ' else 'ㅏ', l_last[2]) + r
            if self.is_stem(l_stem):
                candidates.add((l_stem, r_canon))

        ## Pre-defined set
        if (l, r) in self._predefined:
            for stem in self._predefined[(l, r)]:
                candidates.add(stem)

        return candidates

In [2]:
stems = {
    '깨닫', '가', # ㄷ 불규칙
    '구르', '들르', # 르 불규칙
    '더럽',  '곱', '감미롭', # ㅂ 불규칙 (1)
    '이', '하', '푸르', # # 어미의 첫글자가 종성일 경우
    '낫', '긋', '벗', # ㅅ 불규칙
    '푸', '주', '누', # 우 불규칙
    '오', '' # 오 불규칙 (가제, 규칙 활용 ㅗ + ㅏ = ㅘ)
    '끄', '트', # ㅡ 탈락 불규칙
    '파랗', '하얗', '그렇', '시퍼렇', '노랗', # ㅎ (탈락) 불규칙
    '다하', # 여 불규칙 활용 (2)
}

testset = [
    '깨달아', '가고', # ㄷ 불규칙
    '굴러', '구르라니까', '들러', '들렀다', # 르 불규칙     
    '더러워서', '더럽다', '고와', '감미로워서',  # ㅂ 블규칙 (1)
    '입니다', '합니다', '합니까', '한답니다', '할껄', '있어요', '푸른', # 어미의 첫글자가 종성일 경우
    '나았어', '그어버려', '벗어던져',  # ㅅ 불규칙
    '퍼갔어', '줬습니다', '눴어', # 우 불규칙
    '왔다', # 오 불규칙
    '껐다', '껐어', '텄어', # ㅡ 탈락 블규칙
    '파란', '파라면', '하얀', '노란', # ㅎ (탈락) 불규칙
    '파랬다', '그래', '그랬다', '그랬지', '시퍼렜다', #ㅎ (축약) 불규칙
    '했다', '했었다', '다했다', # 여 불규칙활용 (2)
]

lemmatizer = Lemmatizer(stems = stems)

for word in testset:
    candidates = lemmatizer.candidates(word)
    print('{} : {}'.format(word, candidates))

깨달아 : {('깨닫', '아')}
가고 : {('가', '고')}
굴러 : {('구르', '어')}
구르라니까 : {('구르', '라니까')}
들러 : {('들르', '어')}
들렀다 : {('들르', '었다')}
더러워서 : {('더럽', '어서')}
더럽다 : {('더럽', '다')}
고와 : {('곱', '아')}
감미로워서 : {('감미롭', '어서')}
입니다 : {('이', 'ㅂ니다')}
합니다 : {('하', 'ㅂ니다')}
합니까 : {('하', 'ㅂ니까')}
한답니다 : {('하', 'ㄴ답니다')}
할껄 : {('하', 'ㄹ껄')}
있어요 : {('이', 'ㅆ어요')}
푸른 : {('푸르', 'ㄴ'), ('푸', '른')}
나았어 : {('낫', '았어')}
그어버려 : {('긋', '어버려')}
벗어던져 : {('벗', '어던져')}
퍼갔어 : {('푸', '어갔어')}
줬습니다 : {('주', '었습니다')}
눴어 : {('누', '었어')}
왔다 : {('오', '았다')}
껐다 : {('끄', '었다')}
껐어 : {('끄', '었어')}
텄어 : {('트', '었어')}
파란 : {('파랗', 'ㄴ')}
파라면 : {('파랗', '면')}
하얀 : {('하얗', 'ㄴ'), ('하', '얀')}
노란 : {('노랗', 'ㄴ')}
파랬다 : {('파랗', '았다')}
그래 : {('그렇', '아')}
그랬다 : {('그렇', '았다')}
그랬지 : {('그렇', '았지')}
시퍼렜다 : {('시퍼렇', '었다')}
했다 : {('하', '았다')}
했었다 : {('하', '았었다')}
다했다 : {('다하', '았다')}


In [3]:
candidates = lemmatizer.candidates('그으세요')
print('{} : {}'.format(word, candidates))

다했다 : {('긋', '으세요')}
